In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
path = '/gdrive/My Drive/Lemalabs/Data/'

In [ ]:
Movie_Ratings = pd.read_csv(path+'User_Movie_Ratings.csv')
Movie_Ratings.head()

,Movie Titles,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,Toy Story (1995),5,4,0,0,4,4,0,0,0,4,0,0,3,0,1,5,4,5,0,3,5,0,5,0,5,3,0,0,0,0,0,0,0,0,0,0,0,5,0,...,0,0,0,5,0,0,4,0,0,2,0,0,4,3,3,4,0,3,5,3,5,0,0,5,0,3,3,0,4,3,2,3,4,0,4,0,0,5,0,0
1,GoldenEye (1995),3,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,5
2,Four Rooms (1995),4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,3,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0
3,Get Shorty (1995),3,0,0,0,0,0,5,0,0,4,0,5,5,0,0,5,0,3,4,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0,0,0,0,0,2,0,0,0
4,Copycat (1995),3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0,0,0,3,0,0,4,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Has_Rated = pd.read_csv(path+'Did_User_Rate_Movie.csv')
Has_Rated.head()

,Movie Title,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,Toy Story (1995),1,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,0,1,1,1,1,1,0,1,0,0,1,0,0
1,GoldenEye (1995),1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,Four Rooms (1995),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,Get Shorty (1995),1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0
4,Copycat (1995),1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
num_movies = Movie_Ratings.shape[0]
num_users = Movie_Ratings.shape[1] - 1
print('Total Number of Movies = {}'.format(num_movies))
print('Total Number of Users = {}'.format(num_users))

Total Number of Movies = 1682
Total Number of Users = 943


Assume the number of Genres is 10

In [ ]:
Genres = 10

To find the movie Genre, we need to create movie paramters of the shape -- > Number of movies * Number of Genre

In [ ]:
X =np.random.random(size=(num_movies,Genres))
X.shape

(1682, 10)

In [ ]:
Xparam_num = X.size  #Total Number of X parameters
Xparam_num

16820

To find what kind of movies the user is interested in, we need to create user paramters of the shape -- > Number of users * Number of Genre

In [ ]:
Theta =np.random.random(size=(num_users,Genres))
Theta.shape

(943, 10)

Combining both prameters

In [ ]:
params = np.concatenate((np.ravel(X), np.ravel(Theta)))

In [ ]:
params.shape

(26250,)

X* theta.T must give us the expected ratings that the user will give.

Recommendation systems is generally used to recommend movie for which the user will give high ratings so lets take the movie ratings as Y.

In [ ]:
Y=Movie_Ratings.drop(columns=['Movie Titles'])
Y.head()

,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,User 40,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,5,4,0,0,4,4,0,0,0,4,0,0,3,0,1,5,4,5,0,3,5,0,5,0,5,3,0,0,0,0,0,0,0,0,0,0,0,5,0,0,...,0,0,0,5,0,0,4,0,0,2,0,0,4,3,3,4,0,3,5,3,5,0,0,5,0,3,3,0,4,3,2,3,4,0,4,0,0,5,0,0
1,3,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,5
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,3,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0
3,3,0,0,0,0,0,5,0,0,4,0,5,5,0,0,5,0,3,4,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0,0,0,0,0,2,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0,0,0,3,0,0,4,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y.shape

(1682, 943)

In [ ]:
R=Has_Rated.drop(columns=['Movie Title'])
R.head()

,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,User 40,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,1,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,0,1,1,1,1,1,0,1,0,0,1,0,0
1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Our next step is to add regularization to both the cost and gradient calculations. We'll create one final regularized version of the function (note that this version includes an additional regularisation rate parameter called "lambda").

In [ ]:
def cost(params, Y, R, Genres, reg_rate):
    Y = np.matrix(Y)  # (1682, 943)
    R = np.matrix(R)  # (1682, 943)
    num_movies = Y.shape[0]
    num_users = Y.shape[1]
    
    X_params = params[:Xparam_num]
    theta_params = params[Xparam_num:]

    # reshape the parameter array into parameter matrices
    X = np.matrix(np.reshape(X_params, (num_movies, Genres)))  # (1682, 10)
    Theta = np.matrix(np.reshape(theta_params, (num_users, Genres)))  # (943, 10)
        
    # compute the cost
    error = np.multiply((X * Theta.T) - Y, R)  # (1682, 943)
    squared_error = np.power(error, 2)  # (1682, 943)
    J = (1. / 2) * np.sum(squared_error)
    
    # add the cost regularization
    J = J + ((reg_rate / 2) * np.sum(np.power(Theta, 2)))
    J = J + ((reg_rate / 2) * np.sum(np.power(X, 2)))
    
    
    # initializations
    X_grad = np.zeros(X.shape)  # (1682, 10)
    Theta_grad = np.zeros(Theta.shape)  # (943, 10)
    
    # calculate the gradients with regularization
    X_grad = (error * Theta) + (reg_rate * X)
    Theta_grad = (error.T * X) + (reg_rate * Theta)
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(X_grad), np.ravel(Theta_grad)))
    
    return J, grad

In [ ]:
J, grad = cost(params, Y, R, Genres, 3.5)    #Testing the Cost function

In [ ]:
reg_rate = 3.5
from scipy.optimize import minimize

fmin = minimize(fun=cost, x0=params, args=(Y, R, Genres, reg_rate), method='CG', jac=True, options={'maxiter': 10000})
fmin

     fun: 43338.21691669744
     jac: array([-4.51554454e-06, -5.40081360e-06, -5.42092255e-06, ...,
        6.16741858e-07,  4.42522251e-07,  8.75599939e-07])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 6155
     nit: 4057
    njev: 6154
  status: 2
 success: False
       x: array([ 1.02026265,  0.86466958,  0.84207729, ..., -0.56967555,
        0.80976943,  1.36907719])

In [ ]:
X = np.matrix(np.reshape(fmin.x[:Xparam_num], (num_movies, Genres)))
Theta = np.matrix(np.reshape(fmin.x[Xparam_num:], (num_users, Genres)))

X.shape, Theta.shape

((1682, 10), (943, 10))

In [ ]:
R = (R != 0)
Pred_Recommendations = round(np.multiply((X * Theta.T), R), 0)
Pred_Recommendations.shape

(1682, 943)

In [ ]:
Pred_Recommendations= np.asarray(Pred_Recommendations)

In [ ]:
Pred_Recommendations[:,0]

array([4., 3., 3., ..., 0., 0., 0.])

In [ ]:
Movie_Ratings['User 1']

0       5
1       3
2       4
3       3
4       3
       ..
1677    0
1678    0
1679    0
1680    0
1681    0
Name: User 1, Length: 1682, dtype: int64

In [ ]:
Y.shape[1]

943

In [ ]:
from sklearn.metrics import accuracy_score

avg_acc=[]

for i in range(Y.shape[1]):
  acc = accuracy_score(Movie_Ratings['User '+str(i+1)], Pred_Recommendations[:,i])
  avg_acc.append(acc)
  print('Score of {}th User: {} %'.format(i+1, round(acc*100),2))

Score of 1th User: 92.0 %
Score of 2th User: 99.0 %
Score of 3th User: 98.0 %
Score of 4th User: 100.0 %
Score of 5th User: 94.0 %
Score of 6th User: 94.0 %
Score of 7th User: 87.0 %
Score of 8th User: 98.0 %
Score of 9th User: 99.0 %
Score of 10th User: 97.0 %
Score of 11th User: 94.0 %
Score of 12th User: 99.0 %
Score of 13th User: 82.0 %
Score of 14th User: 97.0 %
Score of 15th User: 97.0 %
Score of 16th User: 96.0 %
Score of 17th User: 99.0 %
Score of 18th User: 92.0 %
Score of 19th User: 99.0 %
Score of 20th User: 98.0 %
Score of 21th User: 95.0 %
Score of 22th User: 96.0 %
Score of 23th User: 96.0 %
Score of 24th User: 98.0 %
Score of 25th User: 99.0 %
Score of 26th User: 98.0 %
Score of 27th User: 99.0 %
Score of 28th User: 97.0 %
Score of 29th User: 99.0 %
Score of 30th User: 99.0 %
Score of 31th User: 99.0 %
Score of 32th User: 99.0 %
Score of 33th User: 100.0 %
Score of 34th User: 99.0 %
Score of 35th User: 99.0 %
Score of 36th User: 99.0 %
Score of 37th User: 98.0 %
Score of

In [ ]:
Average_accuracy = sum(avg_acc)/Y.shape[1]
print('Average Accuracy in predictions for all users : {} %'.format(round(Average_accuracy*100),2))

Average Accuracy in predictions for all users : 97.0 %


### **Predictions for a new User**

In [ ]:
Movie_Ratings['Movie Titles'][0]

'Toy Story (1995)'

### **Add some new ratings for the new user**

In [ ]:
new_ratings = np.zeros((1682, 1))

new_ratings[0] = 4
new_ratings[6] = 3
new_ratings[11] = 5
new_ratings[53] = 4
new_ratings[63] = 5
new_ratings[65] = 3
new_ratings[68] = 5
new_ratings[97] = 2
new_ratings[182] = 4
new_ratings[225] = 5
new_ratings[354] = 5

print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][0], str(int(new_ratings[0]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][6], str(int(new_ratings[6]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][11], str(int(new_ratings[11]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][53], str(int(new_ratings[53]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][63], str(int(new_ratings[63]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][65], str(int(new_ratings[65]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][68], str(int(new_ratings[68]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][97], str(int(new_ratings[97]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][182], str(int(new_ratings[182]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][225], str(int(new_ratings[225]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][354], str(int(new_ratings[354]))))

Rated Toy Story (1995) with 4 stars.
Rated Twelve Monkeys (1995) with 3 stars.
Rated Usual Suspects, The (1995) with 5 stars.
Rated Outbreak (1995) with 4 stars.
Rated Shawshank Redemption, The (1994) with 5 stars.
Rated While You Were Sleeping (1995) with 3 stars.
Rated Forrest Gump (1994) with 5 stars.
Rated Silence of the Lambs, The (1991) with 2 stars.
Rated Alien (1979) with 4 stars.
Rated Die Hard 2 (1990) with 5 stars.
Rated Sphere (1998) with 5 stars.


### **We can add this custom ratings vector to the data set so it gets included in the model.**

In [ ]:
Y = np.append(Y, new_ratings, axis=1)
R = np.append(R, new_ratings != 0,  axis=1)

We're now ready to train the collaborative filtering model. We're going to run the optimization routine using our cost function, parameter vector, and data matrices at inputs.

In [ ]:
Y.shape, R.shape

((1682, 944), (1682, 944))

In [ ]:
num_movies = Y.shape[0]
num_users = Y.shape[1]


X = np.random.random(size=(num_movies, Genres))
Theta = np.random.random(size=(num_users, Genres))

params = np.concatenate((np.ravel(X), np.ravel(Theta)))

X.shape, Theta.shape

((1682, 10), (944, 10))

In [ ]:
reg_rate = 3.5
from scipy.optimize import minimize

fmin = minimize(fun=cost, x0=params, args=(Y, R, Genres, reg_rate), method='CG', jac=True, options={'maxiter': 10000})
fmin

     fun: 43352.85265305669
     jac: array([ 2.99738887e-07,  8.40029379e-07,  4.08027415e-07, ...,
        3.93504155e-07,  4.27140285e-07, -6.75215619e-07])
 message: 'Optimization terminated successfully.'
    nfev: 3558
     nit: 2356
    njev: 3558
  status: 0
 success: True
       x: array([ 0.37602979,  0.20386065,  0.42878742, ...,  0.13466171,
        0.6703571 , -0.10767538])

In [ ]:
fmin.x.shape

(26260,)

In [ ]:
Xparam_num

16820

In [ ]:
X = np.matrix(np.reshape(fmin.x[:Xparam_num], (num_movies, Genres)))
X.shape

(1682, 10)

In [ ]:
Theta = np.matrix(np.reshape(fmin.x[Xparam_num:], (num_users, Genres)))

Theta.shape

(944, 10)

In [ ]:
R = (R != 0)
Pred_Recommendations = np.multiply((X * Theta.T), R)
Pred_Recommendations.shape

(1682, 944)

In [ ]:
R = (R != 1)
New_Recommendations = np.multiply((X * Theta.T), R)
my_recs = New_Recommendations[:, 943]

In [ ]:
my_recs.shape

(1682, 1)

In [ ]:
idx = np.argsort(my_recs, axis=0, )[::-1]
print(idx)

[[ 49]
 [180]
 [173]
 ...
 [ 97]
 [354]
 [  0]]


In [ ]:
idx.shape

(1682, 1)

In [ ]:
print("Top 10 movie predictions:")
for i in range(20):
    j = int(idx[i])
    print('Predicted rating of {0} for movie {1}.'.format(str((round(float(my_recs[j]),1))),Movie_Ratings['Movie Titles'][j]))

Top 10 movie predictions:
Predicted rating of 4.9 for movie Star Wars (1977).
Predicted rating of 4.8 for movie Return of the Jedi (1983).
Predicted rating of 4.8 for movie Raiders of the Lost Ark (1981).
Predicted rating of 4.7 for movie Empire Strikes Back, The (1980).
Predicted rating of 4.7 for movie Titanic (1997).
Predicted rating of 4.5 for movie Terminator 2: Judgment Day (1991).
Predicted rating of 4.5 for movie Terminator, The (1984).
Predicted rating of 4.4 for movie Braveheart (1995).
Predicted rating of 4.3 for movie Indiana Jones and the Last Crusade (1989).
Predicted rating of 4.2 for movie Back to the Future (1985).
Predicted rating of 4.2 for movie Die Hard (1988).
Predicted rating of 4.2 for movie Fugitive, The (1993).
Predicted rating of 4.1 for movie Independence Day (ID4) (1996).
Predicted rating of 4.1 for movie As Good As It Gets (1997).
Predicted rating of 4.1 for movie Jurassic Park (1993).
Predicted rating of 4.1 for movie Star Trek: The Wrath of Khan (1982).
